# WELCOME TO THE ORCA NOTEBOOK
## *Moorena bouillonii* Chemogeography Expansion

This branch of ORCA was used for the 4th Chapter of my PhD Dissertation- *Moorena bouillonii* Chemogeography: Distributional Patterns of Compounds and Compound Families at Multiple Spatial Scales.
Please excuse the digital dust - the whirlwind of finishing my PhD has not yet allowed for me to get everything cleaned up yet. Feel free to check out the code, but if you would like a more user-friendly experience, please see the master branch!
_______________________________________________________________________________________________________________________________

Pipeline for the **O**bjective **R**elational **C**omparative **A**nalyses of mass spectral data, along with other data sources. All you need is a directory of mzXML files or a features vs samples bucket table to get started!

To run cells of code, select cell and then press **Shift + Enter**. The first cell loads-in python modules necessary for the rest of the code to function

In some of the cells below, the user will need to input information, such as setting paths, setting parameters, etc. In these cases, the user will see a cell with variables to be set at the top, followed by a line of '###', below which the rest of the code can be seen. Please set all applicable variables above the '###' line. Tinkering with the code below the '###' line is highly encouraged, as that is precisely why we chose to make it available as a Jupyter Notebook, however it could result in a 'breaking' of ORCA. If that appears to be the case, simply clone again from GitHub to get back to working code.

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from pyteomics import mzxml, auxiliary
import glob
from scipy.spatial.distance import pdist, squareform, cosine
import itertools
import networkx as nx
from statistics import mean
import matplotlib 
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, set_link_color_palette
from collections import Counter
import sklearn.feature_selection
from scipy.stats import ttest_ind
import seaborn as sns; sns.set()


np.set_printoptions(precision=4, edgeitems=5, linewidth=100, suppress=True, threshold=10000)


# MS1 Feature Processing

This cell contains the first variable that you need to assign, if you are using ORCA to do MS1 feature processing - the path to the folder (directory) that contains your samples. If this notebook is in the same folder as your samples, the path should be './'. After you set the path, run the cell (**Shift + Enter**) and it should display a list of your mzXML sample files. If your files are not displayed, your path may be wrong.

Note: If you already have a features vs samples bucket table, and are not trying to use ORCA for MS1 Feature Processing, please proceed to the section in the notebook titled **Analyses**

In [ ]:
path_to_sample_directory = '../Guam_Mb_comps/' #last character should always be '/'

#######################################################################################################

#Load files from the indicated directory with .mzXML extension
fns = glob.glob(path_to_sample_directory +'*.mzXML')
fns.sort()
peak_list = []
peak_range_low = []
peak_range_high = []
rt_list = []
rt_range_low = []
rt_range_high = []
rrt_list = []
rrt_range_low = []
rrt_range_high = []
integ = []
sample = []
fns

### Load metadata for mzXML files
This cell contains another variable for you to assign - the path to a metadata file, that has information about your samples. At minimum, this file needs a column titled 'filename' that includes the filename (including the .mzXML extension) for each of your samples. If this notebook is in the same folder as your metadata file, the path should be './*name-of-metadata-file.csv*'. After you set the path, run the cell and it should display your metadata file as a table, with two additional columns added that include modified versions of the filenames - these are important for mapping metadata later on in the analyses.

In [ ]:
path_to_metadata = '../Guam_Mb_comps/metadata.csv'

##############################################################################################################
#load in metadata file, mapping file names to other information

md = pd.read_csv(path_to_metadata)
md['filename_stripped'] = [f.split('.mzXML')[0] for f in md['filename']]
md['filename_path'] = [path_to_sample_directory + f for f in md['filename']]
md

### Custom retention time function
The original intent behind the design of ORCA was to be able to perform MS1 analyses on heterogeneous datasets (e.g. comparing data files acquired at different time points, under different chromatographic conditions, etc). In particular, we were faced with comparing current samples and legacy samples that had been run on similar but not the same chromatgraphic method (namely, a 2-min increase in run length to compensate for a XX decrease in flow-rate). This meant that using retention time as a second dimension (along with m/z) to charcaterize MS1 features did not allow of the proper consolidation of MS1 features across different samples. We were able to overcome this by instead using relative retention, along with applying an appropriate relative retention time tolerance window as determined through manual validation.

Datasets that are more chromatographically hterogeneous than ours will likely require more complex functions applied to retention time in order for it to remain a reasonable and reliable second dimension by which MS1 features can be consolidated. We provide the opportunity to apply custom retention time functions in the cell below. Some ideas are: XX

As is always the case when setting parameters or applying custom functions in tools, it is the user's responsibility to manually validate that appropriate parameters and functions have been applied. What this specifically means is checking that features known to be the same across different samples are in fact being considered the same in the resultant features vs samples bucket table. If this is not the case, please change the parameters/functions. The best guidance on selecting appropriate parameters and functions comes from studying and identifying the nuances in your dataset!

In [ ]:
#Custom retention time function

def custom_rt(rt, total_run_time):
    return rt/total_run_time

    

### Set paramters for MS1 feature picking

Here are descriptions and recommendations for the different parameters to be set

**bin_width**: The first step in generating feature lists for each sample is to divide the range of m/z values to be considered into bins, sorting features into these bins based on m/z, and then grouping peaks within each bin based on those peaks occurring in consecutive MS1 scans. Bin width determines the size of these bins. Appropriately setting bin width depends on the resolution of the instrument(s) from which data was acquired, and is essentially a form or m/z tolerance. For low resolution instruments/heterogenous sample sets, 1 is a safe starting point.

**bin_offset**: This parameter is unnecessary for most uses, and so should defaultly be set to 0. It is useful in cases where a systematic bias in m/z is seen in the data, e.g. if a feature that should have an m/z of 100.0 is consistently being recorded as 100.3, a bin_offset of 0.3 may be appropriate.

**bins_start**: The lowest m/z to be considered.

**bins_end**: The highest m/z to be considered.

**peak_consecutivity**: This parameter designates how 'consecutive' peaks need to be in order to be considered as composing the same feature. A value of 0 means that peaks will only be considered as part of a singular feature if there are 0 MS1 scans between them in which the peak does not appear. A value of 1 means that if a peak does not appear in one MS1 scan, but it does appear in the scans directly before and after, those instances can will still be grouped together as one feature. For guidedance in setting this parameter, take a look at your data - do peaks tend to occur in consecutive MS1 scans, or are there sometimes gaps?

**peak_cluster_size_cutoff**: This parameter designates how wide (e.g. how many consecutive scans it is found across) a potential feature needs to be in order to be considered a true feature. For example, setting this parameter to 3 means that a potential feature must be recorded in three or more scans meeting the consecutivity criyteria designated via the peak_consecutivity parameter in order to be considered a true feature.

**min_integral**: Minimum integral a potential feature must have in order to be considered a true feature.

**rt_setting**: This parameter determines what is used as the second dimension, along with m/z, is used for defining specific features. The options are 'raw' for using regular retention time, 'relative' for using relative retention time, or 'custom' for applying a custom retention time function that the user can define above.


In [ ]:
#Enter MS1 feature picking paramaters:
bin_width = 1
bin_offset = 0 
bins_start = 200
bins_end = 2000
peak_consecutivity = 0
peak_cluster_size_cutoff = 3
min_integral = 100000

rt_setting = 'raw'

##########################################################################################################################
#From each file, bin and integrate peaks from all MS1 scans
if rt_setting not in ['relative', 'raw', 'custom']:
    print('rt_setting is not properly set. The options are "raw", "relative", or "custom". Please try again.')
    raise SystemExit(0)
for file_name in fns:
    print(file_name)
    rt = []
    run_length = []
    mz = []
    inten = []
    with mzxml.read(file_name) as reader:
        for spectrum in reader:
            run_length.append([spectrum['retentionTime']])
            if spectrum['msLevel'] == 1:
                rt.extend([spectrum['retentionTime']] * len(spectrum['m/z array']))
                mz.extend(spectrum['m/z array'])
                inten.extend(spectrum['intensity array'])
    run_length = np.max(run_length)            
    rt = np.array(rt, dtype = np.float32)
    rt_unique = np.unique(rt)
    rt_rank = pd.factorize(rt)[0]
    mz = np.array(mz, dtype = np.float32)
    inten = np.array(inten, dtype = np.float32)
    
    bins = np.arange(bins_start, bins_end, bin_width)
    for b in bins:
        rt_positions = rt_rank[np.where((mz > (b - (bin_width/2) + bin_offset)) * (mz < (b + (bin_width/2) + bin_offset)))[0]]
        rt_positions = np.array(list(set(rt_positions)))
        #cluster rt_positions when they are consecutive
        clusters = list(np.split(rt_positions, np.where(np.diff(rt_positions) > peak_consecutivity+1)[0]+1))
        cluster_size = np.array([len(c) for c in clusters])
        clusters_filt = [clusters[i] for i in list(np.where(cluster_size > peak_cluster_size_cutoff)[0])]
        for y in clusters_filt:
            idx = np.where((mz > (b - (bin_width/2) + bin_offset )) * (mz < (b + (bin_width/2) + bin_offset )) * (rt_rank >= np.min(y)) * (rt_rank <= np.max(y)))[0]
            xcoord = rt[idx]
            ycoord = inten[idx]
            integral = np.trapz(y = ycoord, x = xcoord)
            if integral > min_integral:
                integ.append(integral)
                peak_list.append(np.mean(mz[idx]))
                peak_range_low.append(np.min(mz[idx]))
                peak_range_high.append(np.max(mz[idx]))
                rt_list.append(np.mean(rt[idx]))
                rt_range_low.append(np.min(rt[idx]))
                rt_range_high.append(np.max(rt[idx]))
                sample.append(file_name)
                if rt_setting == 'relative':
                    rrt_list.append(np.mean(rt[idx])/run_length)
                    rrt_range_low.append(np.min(rt[idx]/run_length))
                    rrt_range_high.append(np.max(rt[idx]/run_length))
                elif rt_setting == 'raw':
                    rrt_list.append(np.mean(rt[idx]))
                    rrt_range_low.append(np.min(rt[idx]))
                    rrt_range_high.append(np.max(rt[idx]))
                elif rt_setting == 'custom':
                    rrt_list.append(custom_rt(np.mean(rt[idx]), run_length))
                    rrt_range_low.append(custom_rt(np.min(rt[idx]), run_length))
                    rrt_range_high.append(custom_rt(np.max(rt[idx]), run_length))
                else:
                    print('rt_setting is not properly set. The options are "raw", "relative", or "custom". Please try again.')
                    raise SystemExit(0)
#         print('current bin: ' + str(b))
#         print('current length of sample: ' + str(len(sample)))
#         print('current length of rrt_list: ' + str(len(rrt_list)))
        
        
    

### Set paramters for MS1 feature consolidation

Here are descriptions and recommendations for the different parameters to be set

**rrt_tolerance**: This is the maximum raw/relative/custom retention time distance between the edges of two features in order for them to be candidates for consolidation.

**bin_width**: Bin width, again, determines the size of m/z feature bins. Appropriately setting bin width depends on the resolution of the instrument(s) from which data was acquired, and is essentially a form or m/z tolerance. For low resolution instruments/heterogenous sample sets, 1 is a safe starting point.

**bin_offset**: This parameter is unnecessary for most uses, and so should defaultly be set to 0. It is useful in cases where a systematic bias in m/z is seen in the data, e.g. if a feature that should have an m/z of 100.0 is consistently being recorded as 100.3, a bin_offset of 0.3 may be appropriate.

**bins_start**: The lowest m/z to be considered.

**bins_end**: The highest m/z to be considered.


In [ ]:
#Add feature consolidation parameters
rrt_tolerance = 0.5 #maximum rrt distance between edges of two feature peaks in order for them to be candidates for consolidation
bin_width = 1
bin_offset = 0 #must be less than half of bin_width
bins_start = 200
bins_end = 2000


################################################################################
#Peak consoslidating

masses = np.arange(bins_start, bins_end, bin_width)
peak_list = np.array(peak_list, dtype = np.float32)
rrt_list = np.array(rrt_list, dtype = np.float32)
rrt_range_high = np.array(rrt_range_high, dtype = np.float32)
rrt_range_low = np.array(rrt_range_low, dtype = np.float32)

updated_integ = []
updated_peak_list = []
updated_rrt_list = []
updated_rrt_range_low = []
updated_rrt_range_high = []
updated_sample = []
temp_peaks = []

#search through each mass, and collapse peaks across samples, based on rrt tolerance/overlap
for mass in masses:
    print(mass)
    idx = list(np.where((peak_list > (mass - (bin_width/2) + bin_offset)) * (peak_list < (mass + (bin_width/2) + bin_offset)))[0])
    i_record = []
    for i in idx:
        temp_peaks_flag = 0
        for j in idx:
            if j != i:
                if sample[i] != sample[j] and max(rrt_range_low[i],rrt_range_low[j]) < min(rrt_range_high[i],rrt_range_high[j]) + rrt_tolerance:
                    temp_peaks_flag += 1
                    temp_peaks.append([i,j])

        i_record.append(i)
        if temp_peaks_flag == 0:
            updated_integ.append(integ[i])
            updated_peak_list.append(peak_list[i])
            updated_rrt_list.append(rrt_list[i])
            updated_rrt_range_low.append(rrt_range_low[i])
            updated_rrt_range_high.append(rrt_range_high[i])
            updated_sample.append(sample[i])
            

for tp in temp_peaks:
    tp.sort()
    
temp_peaks.sort()
temp_peaks = list(tp for tp,_ in itertools.groupby(temp_peaks))

#collapse peaks by graphing overlaps
G = nx.Graph()
G.add_edges_from(temp_peaks)
apeaks = list(nx.connected_components(G))

#update peak lists to include adjusted mz and rrt for collapsed peaks
for apeak in apeaks:
    apeak = list(apeak)
    curr_integ = [integ[a] for a in apeak]
    curr_peak_list = [peak_list[a] for a in apeak]
    curr_rrt_list = [rrt_list[a] for a in apeak]
    curr_rrt_range_low = [rrt_range_low[a] for a in apeak]
    curr_rrt_range_high  = [rrt_range_high[a] for a in apeak]
    curr_sample = [sample[a] for a in apeak]
    for file_name in list(set(curr_sample)):
        csidx = [i for i, j in enumerate(curr_sample) if j == file_name]
        updated_integ.append(sum([curr_integ[c] for c in csidx]))
        updated_peak_list.append(np.mean(curr_peak_list))
        updated_rrt_list.append(np.mean(curr_rrt_list))
        updated_rrt_range_low.append(np.mean(curr_rrt_range_low))
        updated_rrt_range_high.append(np.mean(curr_rrt_range_high))
        updated_sample.append(file_name)
        
                
    

### Generate bucket table
This next cell organizes features into a features vs samples bucket table, to which analyses can then be applied.

In [ ]:
#generate empty Pandas dataframe, to be populated as a bucket table
columns = fns
index0 = list(set(list(zip(updated_peak_list, updated_rrt_list))))
index0.sort()
df_updated = pd.DataFrame(index=index0, columns=columns)
df_updated = df_updated.fillna(0)

#populate bucket table
updated_sample = np.array(updated_sample)
updated_integ = np.array(updated_integ)
for mz_rrt in index0:
    for filename in fns:
        idx_to_df = np.where((updated_peak_list == mz_rrt[0]) * (updated_rrt_list == mz_rrt[1]) * (updated_sample == filename))[0]
        if len(idx_to_df) == 1:
            value = updated_integ[idx_to_df]
            df_updated.at[mz_rrt, filename] = value

df_updated.index = pd.MultiIndex.from_tuples(index0, names=['mz', 'rrt'])
df_updated

In [ ]:
np.sum(df_updated > 0)

### ~ End of MS1 Feature Processing ~

# Analyses

### Load-in features vs samples bucket table

If you already have a features vs samples bucket table, you can load it into ORCA to perform analyses here. Input the path to your metadata table in the cell below. It should be either in tsv or csv format. If you load-in your bucket table and see that it includes columns or rows that it should not, add those columns or rows to a list under the variable **drop_columns** or **drop_rows**, respectively. By setting **col_ref** and/or **row_ref** as 'True', an auxilary dataframe will be produced that contains those dropped columns/rows for future reference. If there is prefix or suffix included in the sample names included in the table, you can remove those by adding that string to be removed in **table_name_trim**. 

Note: All of the following code is setup to work on a bucket table that is arranged so that the features are represented by each row, and the samples are represented by each column. If your bucket table is arranged in the alternative fashion (rows = samples, columns = features), please set **transpose_buckettable** to True.
 
**If you used the MS1 Feature Processing to generate a bucket table above, you can skip the following cell.**

In [ ]:
buckettable_path = '../../MZmine-2.53-Windows/Mb_MS_0_quant.csv'
drop_columns = ['Unnamed: 159', 'row ID', 'row m/z', 'row retention time']
col_ref = True
drop_rows = []
row_ref = False
sample_name_trim = ' Peak area'

transpose_buckettable = False

###################################################################################################

if buckettable_path.split('.')[-1] == 'tsv':
    bt = pd.read_table(buckettable_path)
elif buckettable_path.split('.')[-1] == 'csv':
    bt = pd.read_csv(buckettable_path)
else:
    print('Format invalid. File must be csv or tsv.')
    
if len(drop_columns) > 0:
    if col_ref == True:
        col_ref_df = bt[drop_columns]
    bt.drop(drop_columns, axis = 1, inplace=True)

if len(drop_rows) > 0:
    if row_ref == True:
        row_ref_df = bt.iloc[drop_rows]
    bt.drop(drop_rows, axis = 0, inplace=True)

if transpose_buckettable == True:
    bt = bt.T
    
bt.columns = bt.columns.str.replace(sample_name_trim, '')

df_updated = bt.copy(deep=True)
df_updated

### Alternative: Load-in buckettable, normalize to internal reference, and re-export
If one finds themself needing to normalize a buckettable to the internal standard peak, and then re-exporting for use in another tool (e.g. GNPS feature-based molecular network), you can do that here!

**IS_ref_column**: name of column to look at for referencing the internal standard peak/other peak for normalizing against

**IS_ref_value**: the value used to indicate which row is the internal standard/other peak for normalizing against

**export**: Set to 'True' for exporting the normalized table as a csv

In [ ]:
buckettable_path = '../../MZmine-2.53-Windows/Mb_MS_0_gnpsfilt_quant.csv'
drop_columns = ['Unnamed: 159', 'row ID', 'row m/z', 'row retention time']
col_ref = True
drop_rows = []
row_ref = False
sample_name_trim = ''

transpose_buckettable = False

IS_ref_column = 'row ID'
IS_ref_value = 1454

export = True

###################################################################################################

if buckettable_path.split('.')[-1] == 'tsv':
    bt = pd.read_table(buckettable_path)
elif buckettable_path.split('.')[-1] == 'csv':
    bt = pd.read_csv(buckettable_path)
else:
    print('Format invalid. File must be csv or tsv.')
    
IS_index = bt[bt[IS_ref_column] == IS_ref_value].index

if len(drop_columns) > 0:
    if col_ref == True:
        col_ref_df = bt[drop_columns]
    bt.drop(drop_columns, axis = 1, inplace=True)

if len(drop_rows) > 0:
    if row_ref == True:
        row_ref_df = bt.iloc[drop_rows]
    bt.drop(drop_rows, axis = 0, inplace=True)
    
if transpose_buckettable == True:
    bt = bt.T
    
bt.columns = bt.columns.str.replace(sample_name_trim, '')

replace_val = bt.loc[IS_index].T.mean().values[0]
bt.loc[IS_index] = bt.loc[IS_index].replace(0,replace_val)
bt = bt/np.array(bt.loc[IS_index])

## NEED TO FIGURE OUT HOW TO BEST ORGANIZE THIS
df_IS_normed = pd.concat([col_ref_df, bt], axis=1).drop(['Unnamed: 159'], axis = 1)
df_IS_normed.to_csv('../../MZmine-2.53-Windows/Mb_MS_0_gnpsfilt_quant_ISnormed.csv', index=False)


### Load metadata for features vs samples bucket table
This cell contains another variable for you to assign - the path to a metadata file, that has information about the samples in your bucket table. At minimum, this file needs a column with the names of your samples, matching how they are named in the bucket table. This column will be used to map any other metadata to the samples. 

**If you used the MS1 Feature Processing to generate a bucket table above, and you have already loaded-in a metadata table above, you can skip the following cell.**

In [ ]:
path_to_metadata = '../../ReDU_v3.0_Mb_extracts_draft.tsv'

#############################################################################
#load in metadata file, mapping file names to other information
if path_to_metadata.split('.')[-1] == 'tsv':
    md = pd.read_table(path_to_metadata)
elif path_to_metadata.split('.')[-1] == 'csv':
    md = pd.read_csv(path_to_metadata)

md

### Custom transformation for your data
Below, a number of transformations can be applied to your bucket table. If you cannot find a transformation that meets your needs, feel free to create a custom transformation in the cell directly below.

In [ ]:
# Custom transformation
def custom_transform(df_transformed):
    #Edit here#
    return df_transformed

### Transformations
Options for transformations to be applied to your bucket table are log (base 10) and natural log transformations, unit vector normalization, conversion of feature values to proportions per sample, conversion of feature values to presence-absence data, or any custom transformation defined by the user. In order to apply a transformation, set the applicable variable to 'True'.

In [ ]:
# Apply transformations
log_10_transform = True
log_natural_transform = False

unit_vector_normalization = False
transform_to_proportions = False
presence_absence = False

custom_transform = False

#############################################################################
df_transformed = df_updated.copy(deep = True)
flag_transf = 0

if log_10_transform == True and log_natural_transform == True:
    flag_transf = 1
    print('Cannot use both log transforms at the same time.')

#log (base 10) transform integrals
if log_10_transform == True and flag_transf == 0:
    df_transformed[df_transformed < 100] = 1
    df_transformed = np.log10(df_transformed)
    
#log (natural) transform integrals
if log_natural_transform == True and flag_transf == 0:
    df_transformed[df_transformed < 100] = 1
    df_transformed = np.log10(df_transformed)
    
# Unit vector normalization
if unit_vector_normalization == True:
    df_transformed = df_transformed/(df_transformed**2).sum()**0.5
    
# Change integrals to proportions
if transform_to_proportions == True:
    df_transformed = df_transformed/df_transformed.sum()
    
# Change presence-absence (1-0) data
if presence_absence == True:
    df_transformed[df_transformed > 0] = 1
    
    
#Applies custom transform
if custom_transform == True:
    df_transformed = custom(df_transformed)
    
df_transformed
    


## Sample Set Filtering/Additional Preprocessing Options

### Drop features based on certain label (e.g. blanks)
I used this code snippet to remove features associated with different types of blanks in my dataset. 

**filterby**: name of column that contains labels of groups whose features you would like to remove

**label_to_drop**: list of labels for groups whose features should be dropped

**filename_column**: column that contains filenames. These should be filenames in the format that is shard between the buckettable and the metadata dataframe

In [ ]:
# drop features from blanks [indicate process, solvent, qc_mix, and internal_standard seperately  **Order does not matter]
filterby = 'BlankType'
label_to_drop = ['process','solvent','qc_mix','internal_standard']
filename_column = 'filename'

################################################################################################
for blank in label_to_drop:
    query = filterby + ' == "' + blank + '"'
    print(blank)
    print(len(df_transformed[md.query(query)['filename']][df_transformed[md.query(query)['filename']].T.median() > 0]))
    df_transformed.drop(df_transformed[md.query(query)['filename']][df_transformed[md.query(query)['filename']].T.median() > 0].index, axis = 0, inplace=True)

### Remove unwanted samples - method 1
I used this code to remove blanks (after I had used them for removing features that could be found in blanks), so that I could carry on analysis only of samples I was interested in.

**filterby**: name of metadata column that contains labels of samples you would like to remove

**label_to_drop**: label of smaples you would like to drop. The code is setup to keep whatever samples do not have this label.

**filename_column**: column of filenames - this column serves as the identifier that allows linking data from the metadata tabel to the buckettable.

In [ ]:
# remove blanks 
filterby = 'SampleType'
label_to_drop = 'blank_QC'
filename_column = 'filename'

################################################################################################
query = filterby + ' != "' + label_to_drop + '"'
df_transformed = df_transformed[md.query(query)['filename']]
df_transformed

### Remove unwanted samples - method 2
I used this code to remove samples from locations that were outside the range of *M. bouillonii*, so that I could carry on analysis only of samples I was interested in.

**filterby**: name of metadata column that contains labels of samples you would like to remove

**label_to_drop**: label of smaples you would like to drop. The code is setup to drop samples that do have this label.

**filename_column**: column of filenames - this column serves as the identifier that allows linking data from the metadata tabel to the buckettable.

In [ ]:
# remove non Mb (Panama, unknown, Puerto Rico)
filterby = 'ATTRIBUTE_collection_region'
label_to_drop = 'unknown'
filename_column = 'filename'

################################################################################################
query = filterby + ' == "' + label_to_drop + '"'
df_transformed.drop(list(md.query(query)['filename']), axis = 1, inplace=True)
df_transformed

### Remove features not included in both sample duplicates
Even after intensive preprocessing, MS data can still be quite noisey. Some of this noise can be reduced by running samples in duplicate. This code block removes peaks that appear in one duplicate, but not in the other.

***NOTE***: This code is currently set to only work in the situation of duplicates (not triplicate +), and only if the following naming convention is used to differentiate between duplicates:

**Duplicate 1**: SampleName_01_AnythingElseIncluding.Extension

**Duplicate 2**: SampleName_02_AnythingElseIncluding.Extension

In [ ]:
# remove peaks not present in both reps (remove blanks first!!)
cols = list(df_transformed.columns)
samps = list(set([c.split('_0')[0] for c in cols]))
for s in samps:
    print(s)
    for i in df_transformed.index:
        if sum(df_transformed[df_transformed.filter(like=s).columns].loc[i] > 0) != 2:
            df_transformed.at[i, df_transformed.filter(like=s).columns[0]] = 0
            df_transformed.at[i, df_transformed.filter(like=s).columns[1]] = 0


### Buckettable Checkpoint
Removing peaks that are not present in both duplicates, along with other preprocessing and filtering steps, can take a long time. In order to set aside a dataframe copy, in case one wants to return to it after making additional changes, use the below code.

In [ ]:
df_0 = df_transformed.copy(deep=True)

### Retrieve dataframe from checkpoint

In [ ]:
df_transformed = df_0.copy(deep=True)
df_transformed

### Remove sample duplicates from dataset
I used this code after removing peaks not present in both duplicates, making duplicates consistent and essentially identical. Therefore, one of each duplicate could be dropped, as it was not contributing any new information to analyses.

In [ ]:
# remove dups
df_transformed = df_transformed[df_transformed.filter(like='_01').columns]

### Drop empty features
After various going through various filtering steps, the situation can occur where one has features in the table that did not have any observed peaks in the samples currently included in the dataset. This code cleans things up by removing those empty features.

In [ ]:
# drop COMPLETELY empty features
df_transformed.drop(df_transformed[df_transformed.T.sum() < 0.00000000001].index, axis = 0, inplace=True)

### Drop peaks below a certain size

**min_peak**: peaks with area below or equal to this number will be dropped

In [ ]:
# peak cutoff
min_peak = 6.5

##########################################################################
df_transformed = df_transformed.apply(lambda x: np.where(x < 6.5,0,x))

### Drop features based on frequency in dataset

**freq**: list of integers, indicating frequencies to be dropped, e.g. if you have 50 samples and you wish to drop features that are present in all 50 samples, and 50 to the list

In [ ]:
# drop features based on frequency across samples
freq = [49,48]
######################################################
for f in freq:
    df_transformed.drop(df_transformed[np.sum(df_transformed.T > 0) == f].index, inplace=True)

### Drop features not in the top k peaks of samples
If a feature is counted amongst the top k peaks of any sample, it (and all of the peaks associated with that feature, regardless of rank, will be retained)

**k**: number of top peaks to keep for each sample. 

In [ ]:
# drop all features not in the top k peaks for a sample
k = 250
########################################################

index_to_keep = []

for c in df_transformed.columns:
    cutoff = df_transformed[c].sort_values(ascending=False).iloc[k]
    index_to_keep.extend(list(df_transformed[df_transformed[c] > cutoff].index))

index_to_keep = list(set(index_to_keep))
df_transformed = df_transformed.loc[index_to_keep]

### Drop peaks no in the top k peaks, per sample
This will set peaks to 0 that are not in the top k peaks of each sample

**k**: number of top peaks to keep for each sample. 

In [ ]:
# drop all peaks not in top k
k = 20

for c in df_transformed.columns:
    print(np.sum(df_transformed[c] > 0))
    cutoff = df_transformed[c].sort_values(ascending=False).iloc[k]
    df_transformed[c] = df_transformed[c].apply(lambda x: np.where(x <= cutoff,0,x))
    print(np.sum(df_transformed[c] > 0))

### Drop features no in the top k features of the entire dataset
This sums up features across the dataset, and then ranks them, allowing you to drop features not in the top k

**k**: number of features to retain

In [ ]:
# drop features not in top k
k = 150
df_transformed = df_transformed[np.sum(df_transformed.T) > np.sum(df_transformed.T).sort_values(ascending=False).iloc[k]]

### Limit or filter dataset to only include some groups

**limit_category**: metadata column that includes labels for group that you would like to limit the dataset to

**limit**: group label that you would like to limit the dataset to

**dataframe**: dataframe that you would like to limit

In [ ]:
# limit/filter df
limit_category = 'ATTRIBUTE_collection_region'
limit = 'Guam'
dataframe = df_transformed

####################################################################
file_limit = [c for c in dataframe.columns if c in md[md[limit_category] == limit]['filename'].values]
limited = dataframe[file_limit]

## Principle Component Analysis (PCA)

I have found that applying unit vector transform prior to PCA is helpful. Run the cell below to do so.

**dataframe**: name of dataframe to use

**sample_labels**: metadata column including labels to colorize samples by in PCA plot

**ref_for_labels**: metadata column with sample names; used for mapping metadata to buckettable

**x_label**: axis title

**y_label**: axis title

In [ ]:
# unit vector transform prior to PCA
df_transformed = df_transformed/(df_transformed**2).sum()**0.5

In [ ]:
# Build PCA (10 components), and display explained variance ratios
dataframe = df_transformed

##########################################################################

x =dataframe.T

from sklearn.decomposition import PCA
pca = PCA(n_components=10)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, index = dataframe.T.index, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10'])

pca.explained_variance_ratio_

In [ ]:
# Plot
sample_labels = 'ATTRIBUTE_collection_region'
ref_for_labels = 'filename' #should be the column whose contents exactly match the sample names in df_transformed
x_label = 'PC1 [9.7%]'
y_label = 'PC2 [8.4%]'

#######################################################################################################################

dict_labels = pd.Series(md[sample_labels].values, index=md[ref_for_labels]).to_dict()
samplenames = [dict_labels[n] for n in dataframe]
samplenames

principalDf['labels'] = samplenames

groups = principalDf.groupby('labels')

# Plot
fig, ax = plt.subplots()
ax.margins(0.05)
for name, group in groups:
    ax.plot(group.pc1, group.pc2, marker='o', linestyle='', ms=12, label=name)
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel(x_label)
ax.set_ylabel(y_label)

plt.show()

## Feature & Sample Distributions

### Distribution showing frquency of features across samples

In [ ]:
labels, values = zip(*Counter(np.sum(df_transformed.T > 0)).most_common())

indexes = np.arange(len(labels))
width = 1

plt.figure(figsize=(15,8))
plt.bar(indexes, values, width)
plt.xticks(indexes, labels)
plt.title('Feature Frequency Distribution', fontsize=20)
plt.ylabel('# of features', fontsize=18)
plt.yticks(fontsize=14)
plt.xlabel('# of samples in which feature is detected', fontsize=18)
plt.xticks(fontsize=10)
plt.show()

### Samples, arranged by number of features

In [ ]:
labels = np.sum(df_transformed > 0).sort_values(ascending = False).index
values = np.sum(df_transformed > 0).sort_values(ascending = False).values

indexes = np.arange(len(labels))
width = 1

plt.figure(figsize=(15,8))
plt.bar(indexes, values, width)
plt.xticks(indexes, labels)
plt.title('Features per sample', fontsize=20)
plt.ylabel('# of features', fontsize=18)
plt.yticks(fontsize=14)
plt.xlabel('samples', fontsize=18)
plt.xticks(fontsize=10, rotation=90)
plt.show()

### Number of sample-specific features per sample

In [ ]:
labels = np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0).sort_values(ascending = False).index
values = np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0).sort_values(ascending = False).values

indexes = np.arange(len(labels))
width = 1

plt.figure(figsize=(15,8))
plt.bar(indexes, values, width)
plt.xticks(indexes, labels)
plt.title('Sample-specific features per sample', fontsize=20)
plt.ylabel('# of sample-specific feats.', fontsize=18)
plt.yticks(fontsize=14)
plt.xlabel('samples', fontsize=18)
plt.xticks(fontsize=10, rotation=90)
plt.show()

### Plot: Number of features vs max. peak area

In [ ]:
num_feats = np.sum(df_transformed > 0)
max_peaks = np.max(df_transformed)
ax = pd.concat([max_peaks, num_feats], axis=1).plot.scatter(x=0,y=1)
ax.set_ylabel('# of features')
ax.set_xlabel('max. peak area [log10 transf.]')


### Plot: Number of sample-specific features vs number of features

In [ ]:
np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0)

num_feats = np.sum(df_transformed > 0)
num_samp_spec = np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0)
ax = pd.concat([num_feats, num_samp_spec], axis=1).plot.scatter(x=0,y=1)
ax.set_xlabel('# of features')
ax.set_ylabel('# of sample-specific feats.')

### Plot: Number of sample-specific features vs max peak area

In [ ]:
np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0)

max_peaks = np.max(df_transformed)
num_samp_spec = np.sum(df_transformed[np.sum(df_transformed.T > 0) == 1] > 0)
ax = pd.concat([max_peaks, num_samp_spec], axis=1).plot.scatter(x=0,y=1)
ax.set_ylabel('# of sample-specific feats.')
ax.set_xlabel('max. peak area [log10 transf.]')

## Hierarchical Clustering + Dendrogram

### Hierarchical Clustering + Dendrogram
Hierarchical clustering will be performed on the bucket table generated via ORCA MS1 feature processing or on the bucket table that was independently loaded-in - whichever happened most recently.

Here are the variables that must be set:

**title**: The title that you would like to assign to the dendrogram.

**sample_labels**: The name of the column from the metadata table that includes the names/identifiers of your samples that you would like to use in labelling the branches of the dendrogram.

**ref_for_labels**: The name of the column for the metadata table that includes the names/identifiers of your samples as written in the bucket table. It is important that this variable is assigned correctly, as it allows metadata to be mapped properly to your samples.

**metric**: The distance metric used in performing the hierarchical clustering. For more information on options for distance metrics, check out the documentation for SciPy, specifically the pdist function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html#scipy.spatial.distance.pdist

**method**: The clustering method used in performing the hierarchical clustering. For more information on options for distance metrics, check out the documentation for SciPy, specifically the linkage function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage

**color_cutoff**: The distance value below which you wish clusters to be colorized.


In [ ]:
title = 'Hierarchical Clustering of Induction Experiment Samples'
sample_labels = 'ATTRIBUTE_collection_site'
ref_for_labels = 'filename' #should be the column whose contents exactly match the sample names in df_transformed
metric = 'cosine'
method = 'average'
color_cutoff = 0.5
dataframe = df_transformed #usually df_transformed

#######################################################################################################################

dict_labels = pd.Series(md[sample_labels].values, index=md[ref_for_labels]).to_dict()
samplenames = [dict_labels[n] for n in dataframe]

#Clustering linkages followed by dendrogram construction
matplotlib.rcParams['lines.linewidth'] = 3
hc = linkage(dataframe.T, method, metric = metric)
plt.figure(figsize=(15,8))
plt.title(title, fontsize=20)
plt.ylabel(metric + ' distance', fontsize=18)
plt.yticks(fontsize=14)
dendrogram(
    hc,
    color_threshold=color_cutoff,
    above_threshold_color='k',
    leaf_rotation=90,
    leaf_font_size=14,
    labels = samplenames   
)
#set_link_color_palette(['blue', 'red'])
plt.show()


### Cophenetic correlation
The cophenetic correlation coefficient is a measure of how well the linkages established in hierarchical clustering represent the actual calculated distances between pairs of samples. We can use it to judge how well a dendrogram represents to the underlying data.

In [ ]:
#Determining how well the dendrogram represents the cosine distances in the data
ccorr, distances = cophenet(hc, pdist(dataframe.T, metric))
ccorr

### Hierarchical clustering - tabular output
In some instances, it may be useful to have a tabular output of the hierarchical clustering that is visualized via dendrogram above. Generate such a table by running the cell below.

In [ ]:
#Table output of hierarchical clustering
hc_df = pd.DataFrame(hc)
hc_df.columns = ['cluster 1', 'cluster 2', 'distance at merge', 'number of samples in new cluster']
hc_df['new cluster number'] = [c for c in range(len(list(df_transformed.T.index)),len(list(df_transformed.T.index))+len(hc_df))]

c1_alias = []
for c in hc_df['cluster 1']:
    if c < len(list(df_transformed.T.index)):
        c1_alias.append(list(df_transformed.T.index)[int(c)])
    else:
        c1_alias.append('none')
        
c2_alias = []
for c in hc_df['cluster 2']:
    if c < len(list(df_transformed.T.index)):
        c2_alias.append(list(df_transformed.T.index)[int(c)])
    else:
        c2_alias.append('none')


hc_df['cluster 1 alias'] = c1_alias
hc_df['cluster 2 alias'] = c2_alias
hc_df

### Pairwise distance matrix
In some instances, it may also be useful to generate a pairwise distance matrix for the samples in the bucket table. Do so by running the cell below. Note: This will use the same distance metric as specified above in the hierarchical clustering parameters.

In [ ]:
pairwise = pd.DataFrame(squareform(pdist(df_transformed.T, metric)))
pairwise.index = df_transformed.T.index
pairwise.columns = df_transformed.T.index
pairwise

## Investigating core and specific features

### Query for group-specific features that do not occur elsewhere in dataset
These features occur in all samples from group, but not in other samples

**category**: metadata column with labels for group of interest

**interest**: label for group of interest



In [ ]:
# query for presence in selected group vs absences in rest [length of regionally-specific core feats]
category = 'ATTRIBUTE_collection_region_GuamShrimp'
interest = 'Saipan'

####################################################################

files_of_interest = [c for c in df_transformed.columns if c in md[md[category] == interest]['filename'].values]

empty_feats = df_transformed[np.max(df_transformed.drop(files_of_interest, axis=1).T) <= 0].index
core_metab = df_transformed[np.min(df_transformed[files_of_interest].T) > 0].index

query_result = [r for r in core_metab if r in empty_feats]
print(len(query_result))
query_results

### Query for group-specific features that do not occur elsewhere in dataset [from limited dataset]
These features occur in all samples from group, but not in other samples

**limit_category**: metadata column that includes labels for group that you would like to limit the dataset to

**limit**: group label that you would like to limit the dataset to

**category**: metadata column with labels for group of interest

**interest**: label for group of interest

In [ ]:
# query for presence in selected group vs absences in rest
limit_category = 'ATTRIBUTE_collection_region'
limit = 'Guam'

category = 'ATTRIBUTE_collection_site'
interest = 'Finger Reef'

####################################################################
file_limit = [c for c in df_transformed.columns if c in md[md[limit_category] == limit]['filename'].values]
limited = df_transformed[file_limit]

files_of_interest = [c for c in limited.columns if c in md[md[category] == interest]['filename'].values]

df_transformed[np.max(limited.drop(files_of_interest, axis=1).T) <= 0].index
df_transformed[np.min(limited[files_of_interest].T) > 0].index

query_result = [r for r in limited[np.max(limited.drop(files_of_interest, axis=1).T) <= 0].index if r in limited[np.min(limited[files_of_interest].T) > 0].index]
print(len(query_result))
query_results

### Generate table of feature, core, and specific core counts, per group

**category**: metadata column including labels of groups of interest

**cutoff**: proportion of samples per group that must contain feature, in order for it to be counted as core

**dataframe**: source dataframe to draw from

In [ ]:
# generate table with core, specific core, avg + sd of features per sample, # of samples; PER CATEGORY

category = 'ATTRIBUTE_collection_region_GuamShrimp'
cutoff = 1
dataframe = df_transformed

#############################################################################
core_summary_df = pd.DataFrame()
pd.options.display.float_format = "{:,.2f}".format

for cat in md[category].unique():
    files_of_interest = [c for c in dataframe.columns if c in md[md[category] == cat]['filename'].values]
    feats = [np.sum(dataframe[f] > 0) for f in files_of_interest]
    empty_feats = dataframe[np.max(dataframe.drop(files_of_interest, axis=1).T) <= 0].index
    core_metab = dataframe[np.sum(dataframe[files_of_interest].T > 0)/len(files_of_interest) >= cutoff].index
    query_result = [r for r in core_metab if r in empty_feats]

    core_summary_df[cat] = [np.mean(feats), np.std(feats), len(core_metab), len(query_result), len(files_of_interest)]
    
core_summary_df.drop(['na','unknown','Panama', 'Puerto Rico'], axis=1, inplace=True)
core_summary_df = core_summary_df.sort_index(axis=1)
core_summary_df.index = ['mean - feats/sample', 'std - feats/sample', 'core feats', 'specific core feats', 'sample size']
core_summary_df

### Export table as csv

**path**: file path and name for the export

In [ ]:
path = 'cores_summary_table_075.csv'
#######################################################
core_summary_df.to_csv(path)

### Assemble dataframe only including features that are core to at least one group

**category**: metadata column including labels of groups of interest

**cutoff**: proportion of samples per group that must have peaks of a feature in order for it to be counted as core

**dataframe**: source dataframe

In [ ]:
# assemble dataframe of cores [all samples]
category = 'ATTRIBUTE_collection_region_GuamShrimp'
cutoff = 1
dataframe = df_transformed #usually df_transformed

#################################################################
all_core = []

for cat in md[category].unique():
    files_of_interest = [c for c in dataframe.columns if c in md[md[category] == cat]['filename'].values]
    core_metab = dataframe[np.sum(dataframe[files_of_interest].T > 0)/len(files_of_interest) >= cutoff].index
    all_core.extend(core_metab)
    
all_core = list(set(all_core))

cores = dataframe.loc[all_core]
cores

### Generate dataframe indicating which features are core to which groups

***NOTE***: This is needed for generating pairwise table of overlapping core feats
    
**category**: metadata column including labels of groups of interest

**cutoff**: proportion of samples in a group that must have feature peak in order for that feature to count as core

**groups_to_drop**: list of groups to drop, that you would like to exclude from output

In [ ]:
# assemble dataframe of cores [by group]
category = 'ATTRIBUTE_collection_region_GuamShrimp'
cutoff = 0.75
cats_to_drop = ['na','unknown','Panama', 'Puerto Rico']

#################################################################
all_core_df = pd.DataFrame()

for cat in md[category].unique():
    files_of_interest = [c for c in df_transformed.columns if c in md[md[category] == cat]['filename'].values]
    all_core_df[cat] = np.sum(df_transformed[files_of_interest].T > 0)/len(files_of_interest) >= cutoff

all_core_df.drop(groups_to_drop, axis=1, inplace=True)
#all_core_df = all_core_df[np.max(all_core_df.T) >= cutoff]
all_core_df = all_core_df[np.sum(all_core_df.T) > 0]
all_core_df

### Pairwise table of core features overlapping between groups

**count_v_prop**: display counts of overlapping core features 'count' or display proportion of overlapping core features 'prop'

**output**: options for how to present output: 'table' to print datatable, 'export' to produce csv, or 'hetamap' to produce a heatmap

**path**: file path and name for outputted csv, if 'export' is selected for output

In [ ]:
count_v_prop = 'count'
output = 'table'
path = './output.csv'

##############################################################################################
hm_cores = pd.DataFrame(index = all_core_df.columns, columns = all_core_df.columns)
for col1 in all_core_df.columns:
    for col2 in all_core_df.columns:
        shared_core = len([m for m in all_core_df[all_core_df[col1]].index if m in all_core_df[all_core_df[col2]].index])
        hm_cores.at[col1, col2] = shared_core

hm_cores = hm_cores.sort_index(axis=0)        
hm_cores = hm_cores.sort_index(axis=1)
hm_cores = hm_cores.astype(float)
hm_cores_prop = hm_cores/np.max(hm_cores)
hm_cores_prop = hm_cores_prop.astype(float)

if count_v_prop == 'count':
    output_df = hm_cores
elif count_v_prop == 'prop':
    output_df = hm_cores_prop
else:
    print('Error - please set count_v_prop to count or prop.')
if output == 'table':
    display(output_df)
elif output == 'export':
    output_df.to_csv(path)
elif output == 'heatmap':
    sns.heatmap(output_df, annot=False)
else:
    print('Error - please select table, export, or heatmap for output type')

## Rarefation and core-size tracking

### Generate rarefaction curve

**category**: name of metadata column that includes labels for th groups that you would like to have rarefaction curves generated for

In [ ]:
# rarefaction
category = 'ATTRIBUTE_collection_region_GuamShrimp'

#################################################################
rarefaction = pd.DataFrame()
labels = []
samples = []
feats = []

for cat in md[category].unique():
    print(cat)
    files_of_interest = [c for c in df_transformed.columns if c in md[md[category] == cat]['filename'].values]
    comps = []
    for n in range(1, len(files_of_interest)+1):
        comps.extend(list(itertools.combinations(files_of_interest, n)))

    for comp in comps:
        labels.append(cat) 
        samples.append(len(comp))
        feats.append(np.sum(np.sum(df_transformed[list(comp)].T > 0) > 0))
        
rarefaction['labels'] = labels
rarefaction['samples'] = samples
rarefaction['features'] = feats

## Plot rarefaction curve

**zoom**: Set as 'True', for a zoomed in view of the rarefaction curve

**y_min**: If 'zoom' is set to 'True', this will set the minimum value for the y-axis

**y_max**: If 'zoom' is set to 'True', this will set the maximum value for the y-axis

**x_min**: If 'zoom' is set to 'True', this will set the minimum value for the x-axis

**x_max**: If 'zoom' is set to 'True', this will set the maximum value for the x-axis

In [ ]:
# Plot rarefaction curve
zoom = False
y_min = 0
y_max = 800
X_min = 1
x_max = 5

########################################################################

ax = sns.lineplot(x="samples", y="features",
             hue="labels",
             data=rarefaction, estimator='mean', ci='sd')
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
if zoom == True:
    ax.set_ylim(y_min,y_max)
    ax.set_xlim(x_min,x_max)

## Generate curve, showing change in number of core features as sampling increases

**category**: name of metadata column that includes labels for th groups that you would like to have rarefaction curves generated for

**cutoff**: proportion of samples per group that must contain feature in order for it to be counted as 'core'.

In [ ]:
# core-size vs sampling
category = 'ATTRIBUTE_collection_region_GuamShrimp'
cutoff = 1

#################################################################
core_v_samp = pd.DataFrame()
labels = []
samples = []
feats = []


for cat in md[category].unique():
    print(cat)
    files_of_interest = [c for c in df_transformed.columns if c in md[md[category] == cat]['filename'].values]
    comps = []
    for n in range(1, len(files_of_interest)+1):
        comps.extend(list(itertools.combinations(files_of_interest, n)))

    for comp in comps:
        labels.append(cat) 
        samples.append(len(comp))
        feats.append(np.sum(np.sum(df_transformed[list(comp)].T > 0)/len(comp) >= cutoff))
        
core_v_samp['labels'] = labels
core_v_samp['samples'] = samples
core_v_samp['features'] = feats

## Plot core-sampling curve

**zoom**: Set as 'True', for a zoomed in view of the rarefaction curve

**y_min**: If 'zoom' is set to 'True', this will set the minimum value for the y-axis

**y_max**: If 'zoom' is set to 'True', this will set the maximum value for the y-axis

**x_min**: If 'zoom' is set to 'True', this will set the minimum value for the x-axis

**x_max**: If 'zoom' is set to 'True', this will set the maximum value for the x-axis

In [ ]:
# Plot core-size per sample size curve
zoom = True
y_min = 0
y_max = 800
X_min = 1
x_max = 5

########################################################################

ax = sns.lineplot(x="samples", y="features", hue="labels",
             data=core_v_samp, estimator='mean', ci='sd')
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
if zoom == True:
    ax.set_ylim(y_min,y_max)
    ax.set_xlim(x_min,x_max)

## Generate curve, showing change in number of core features as sampling increases, scaled to mean number of features

**category**: name of metadata column that includes labels for th groups that you would like to have rarefaction curves generated for

**cutoff**: proportion of samples per group that must contain feature in order for it to be counted as 'core'.

In [ ]:
# core per mean samps
category = 'ATTRIBUTE_collection_region_GuamShrimp'
cutoff = 1

#################################################################
core_v_samp = pd.DataFrame()
labels = []
samples = []
feats = []
core_vals = []
props = []

for cat in md[category].unique():
    print(cat)
    files_of_interest = [c for c in df_transformed.columns if c in md[md[category] == cat]['filename'].values]
    comps = []
    for n in range(1, len(files_of_interest)+1):
        comps.extend(list(itertools.combinations(files_of_interest, n)))

    for comp in comps:
        labels.append(cat) 
        samples.append(len(comp))
        feats.append(np.sum(np.sum(df_transformed[list(comp)].T > 0) > 0))
        core_vals.append(np.sum(np.sum(df_transformed[list(comp)].T > 0)/len(comp) >= cutoff))
        props.append(np.sum(np.sum(df_transformed[list(comp)].T > 0)/len(comp) >= cutoff)/np.sum(np.sum(df_transformed[list(comp)].T > 0) > 0))
        
core_v_samp['labels'] = labels
core_v_samp['samples'] = samples
core_v_samp['features'] = feats
core_v_samp['core_va']
core_v_samp['proportion'] = props

## Plot core-sampling feature-scaled curve

**zoom**: Set as 'True', for a zoomed in view of the rarefaction curve

**y_min**: If 'zoom' is set to 'True', this will set the minimum value for the y-axis

**y_max**: If 'zoom' is set to 'True', this will set the maximum value for the y-axis

**x_min**: If 'zoom' is set to 'True', this will set the minimum value for the x-axis

**x_max**: If 'zoom' is set to 'True', this will set the maximum value for the x-axis

In [ ]:
# Plot
zoom = False
y_min = 0
y_max = 800
X_min = 1
x_max = 5

###################################################################################
ax = sns.lineplot(x="samples", y="proportion", hue="labels",
             data=core_v_samp, estimator='mean', ci='sd')
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
if zoom == True:
    ax.set_ylim(y_min,y_max)
    ax.set_xlim(x_min,x_max)

## Tests for within group statistically significant differences
This code generates pairwise values that indicate how related each pair of samples is: cosine distance, number of shared features, proportion of shared features (based on the smaller feature count of each pair). These pairwise scores are labelled based on whether the two samples cam from the same group, or different groups. Then, a t-test is used to test the null hypothesis that mean score (cosine distance, number of shared features, proportion of shared features) for pairs within the same groups is not statistically significant for the mean score of pairs from separate groups.

The cell below does the pairwise calculations.

**category**: metadata column indicating the groups that you wish to test

**dataframe**: the name of the dataframe to pull data from.

In [ ]:
# generate pairwise values for cosine distance and number of shared features (prop?)
category = 'ATTRIBUTE_shrimp_v_nonshrimp'
dataframe = limited

########################################################
same_group = []
cos_dist = []
shared_feats = []
sf_prop = []

for pair in list(itertools.combinations(dataframe.columns, 2)):
    same_group.append(md[md['filename'] == pair[0]][category].values[0] == md[md['filename'] == pair[1]][category].values[0])
    cos_dist.append(cosine(dataframe[pair[0]].values, dataframe[pair[1]].values))
    shared_feats.append(np.sum(np.sum(dataframe[[pair[0], pair[1]]].T > 0) == 2))
    sf_prop.append(np.sum(np.sum(dataframe[[pair[0], pair[1]]].T > 0) == 2)/np.min(np.sum(dataframe[[pair[0], pair[1]]] > 0).values))

stats_test = pd.DataFrame(index = same_group)
stats_test['cos_dist'] = cos_dist
stats_test['shared_feats'] = shared_feats
stats_test['sf_prop'] = sf_prop

stats_test
                                  


## T-test of pairwise calculated scores

**measure**:'cos_dist', 'shared_feats', or 'sf_prop'

In [ ]:
# t-test to test if distributions of cosine distance and shared core features differ\
## between two populations (same label vs different label)
measure = 'sf_prop'
###############################################################################################
ttest_ind(stats_test.loc[True][measure], stats_test.loc[False][measure], equal_var=False)


## Feature Selection + Heat Map

### Feature selection
For feature selection, ORCA uses univariate feature selection, as implemented in sklearn (https://scikit-learn.org/stable/). Univariate feature selection was selected as the optimal strategy of feature selection for this particular problem (e.g. identifying the most important features in driving clustering of different groups of samples) as it considers features individually and so is not negatively impacted by correlated features. In order to truly judge the statistical significance of features, univariate feature selection must be applied to a dataset with proper numbers of replicates per category of sample, and must also meet the three assumptions necessary for applying the ANOVA statistical test (here is a good resource for reviewing those assumptions: https://sites.ualberta.ca/~lkgray/uploads/7/3/6/2/7362679/slides_-_anova_assumptions.pdf) Even in cased where the assumptions are not met, or there are not enough replicate samples per category, feature selection can still be helpful for generating hypotheses about which features may be significant. To conduct univariate feature selection, please set the below parameters:

**sample_labels**: Input the column name from the metadata table that you would like samples to be grouped by.

**ref_for_labels**: This should be the column name whose contents exactly match the sample names in df_transformed. This will allow the sample (grouping) labels to be properly mapped to the samples.

**top_k**: Number of top features to display.

**rank_by**: Column to rank results by. This could be 'p-value' or 'F-values', if the goal is to see the most statistically significant features, as determined by univariate feature selection. 

**ascending**: Setting this variable to 'True' will list values from the designated column (as indicated with the 'rank_by' variable) from smallest to largest, while setting it to 'False' will list the values from largest to smallest.



In [ ]:
sample_labels = 'location'
ref_for_labels = 'filename_path' #should be the column whose contents exactly match the sample names in df_transformed

top_k = 5
rank_by = 'Saipan'
ascending = False

#######################################################################################################################

dict_labels = pd.Series(md[sample_labels].values, index=md[ref_for_labels]).to_dict()
samplenames = [dict_labels[n] for n in df_transformed]

df_featselect = df_transformed.copy(deep = True)
df_featselect.index = list(df_transformed.index)

F_values, p_values = sklearn.feature_selection.f_classif(df_featselect.T.values,samplenames)
feat_select_results = pd.DataFrame(index = list(df_transformed.index))
feat_select_results['F-values'] = F_values
feat_select_results['p-values'] = p_values

group_means = df_featselect.T.groupby(md['location'].values).agg('mean').T

fs = pd.concat([feat_select_results, group_means], axis = 1).sort_values(by = rank_by, ascending=ascending).head(top_k)
fs




### Heat map visualizations
Heat maps can be generated for the feature selection output, or for an excerpt of the features vs samples bucket table.

**feat_select_or_bucket_table**: Indicate either 'feat_select' or 'bucket_table', depending on which you would like a heat map generated of.

If you select 'bucket_table', you can also indicate an m/z range for which features you would like included.

**mz_high**: Upper limit of m/z for inclusion in the heat map

**mz_low**: Lower limit of m/z for inclusion in the heat map

In [ ]:
feat_select_or_bucket_table = 'bucket_table'
mz_high = 460
mz_low = 450

##################################################################################################################

if feat_select_or_bucket_table == 'feat_select':
    ax = sns.heatmap(fs.drop(['p-values', 'F-values'], axis=1))
elif feat_select_or_bucket_table == 'bucket_table':
    ax = sns.heatmap(df_transformed.query('mz > @mz_low and mz < @mz_high'))
else:
    print('invalid input. Try "feat_select" or "bucket_table".')
    

## Tentative Identification of Top Features

***NOTE***: This code currently only works for buckettables generated via ORCA MS1 feature processing

While m/z of an MS1 feature is not enough information to confidently identify what compound is in a sample, cross referencing features with lists of known compounds can allow for quick dereplication, which is very helfpul in the context of natural products discovery. The cell below allows one to load in a table of known compounds, their masses, and expected m/z for protenated and sodiated peaks, and then annotate features in your dataset based on this set of known compounds. By default, features are ordered based on which are the most prominent across the entire dataset (e.g. which features have the largest values).

The parameters to set are:

**mz_tolerance**: How close a feature's m/z must be to the protenated or sodiated m/z of a known compound in order for the feature to be annotated as that known compound.

**top_k**: Number of features to output. Note: features are ranked by greattest value to smallest value, as determined by selecting the max value for a feature across all samples.

**db_path**: Path to table of known compounds to be used for annotation. Table must be a csv file with at least three columns: 'Name', 'Protenated Peak', and 'Sodiated Peak'.

In [ ]:
mz_tolerance = 1
top_k = 10
db_path = './Moorea_bouillonii_db.csv'


######################################################################################################################

db = pd.read_csv(db_path)

df_top = df_transformed.T.max().sort_values(ascending=False).head(top_k).reset_index()

df_top.columns = ['mz', 'rt', 'max transformed integral']

column_pids = []
column_difs = []
for m in df_top['mz']:
    putative_ids = []
    difs = []
    for i in db.index:
        if abs(m - db.iloc[i]['Protenated Peak']) <= mz_tolerance:
            putative_ids.append(db.iloc[i]['Name'] + ' [M+H]+')
            difs.append(round(abs(m - db.iloc[i]['Protenated Peak']),2))
    for i in db.index:
        if abs(m - db.iloc[i]['Sodiated Peak']) <= mz_tolerance:
            putative_ids.append(db.iloc[i]['Name'] + ' [M+Na]+')
            difs.append(round(abs(m - db.iloc[i]['Sodiated Peak']),2))
    if len(putative_ids) == 0:
        putative_ids.append('None')
        difs.append(0)
    column_pids.append(putative_ids)
    column_difs.append(difs)
    
df_top['putative ids'] = column_pids
df_top['difference'] = column_difs
df_top

## Generate Feature Priority List
Create consolidated list of top k features in each sample, and query against user-supplied datasheet of known compounds. Queries based on *m/z*.

**k**: number of top features to consider, per sample

**mz_tolerance**: Maximum number of Daltons difference between a feature and known compound compared against, in order for that featrue to be putatively identified as that known compound.

**db_path**: path to user-supplied known compound datasheet (csv format) to compare against. Table must be a csv file with at least three columns: 'Name', 'Protenated Peak', and 'Sodiated Peak'.

**sample_labels**: metadata column to use to label what sample groups a particular top feature came from.

**ref_for_labels** metadata column of sample names. This column will be used to map metadata to samples included in the buckettable of data.

In [ ]:
#aggregate top peaks, show which locations from, and do putative ids
k = 5
mz_tolerance = 0.1
db_path = './data/Moorea_bouillonii_db.csv'
sample_labels = 'ATTRIBUTE_collection_region'
ref_for_labels = 'filename'
#######################################################################################################

indices = []
samples = []
dict_labels = pd.Series(md[sample_labels].values, index=md[ref_for_labels]).to_dict()

for samp in df_transformed.columns:
    indices.extend(list(df_transformed[samp].sort_values(ascending=False).head(k).index))
    samples.extend([samp] * k)

df = pd.DataFrame([indices,samples]).T.sort_values(by=0)
gb = df.groupby(0)[1].unique()
df = pd.concat([col_ref_df, gb], axis=1, join = 'inner')
df.drop(['Unnamed: 159', 'row ID'], axis=1, inplace=True)
df.rename(columns={'row m/z': 'm/z', 'row retention time':'retention time', 1: "samples"}, inplace = True)
locs = []
for s in df['samples']:
    samplenames = [dict_labels[ss] for ss in s]
    locs.append(list(set(samplenames)))
df['source location'] = locs

db = pd.read_csv(db_path)

column_pids = []
column_difs = []
for m in df['m/z']:
    putative_ids = []
    difs = []
    for i in db.index:
        if abs(m - db.iloc[i]['Protenated Peak']) <= mz_tolerance:
            putative_ids.append(db.iloc[i]['Name'] + ' [M+H]+')
            difs.append(round(abs(m - db.iloc[i]['Protenated Peak']),2))
    for i in db.index:
        if abs(m - db.iloc[i]['Sodiated Peak']) <= mz_tolerance:
            putative_ids.append(db.iloc[i]['Name'] + ' [M+Na]+')
            difs.append(round(abs(m - db.iloc[i]['Sodiated Peak']),2))
    if len(putative_ids) == 0:
        putative_ids.append('None')
        difs.append(0)
    column_pids.append(putative_ids)
    column_difs.append(difs)
    
df['putative ids'] = column_pids
df['difference'] = column_difs
df

### Export priority list table as a csv
**path**: path and name for output csv

In [ ]:
path = './priority_list.csv'
##################################################################
df.to_csv(path)

# Miscellaneous 
Here are some bits of code that might be useful. Feel free to add your own!

In [ ]:
#number of features per sample, post transformations
df_transformed[df_transformed > 0].count()

In [ ]:
# determine how many samples different features occur in
np.sum(df_transformed.T > 0).sort_values()

In [ ]:
# features that occur in k samples
k = 1
##########################################################
df_transformed[np.sum(df_transformed.T > 0) == k].index

In [ ]:
# look up m/z and rt for a specific feature
feat_num = 49
####################################################
col_ref_df.loc[feat_num]

In [ ]:
# query to see if feat (esp. singletons) was aligned optimally
## via compare it to other feats with similar m/z

min_mz = 457
max_mz = 458
#######################################

col_ref_df[col_ref_df['row m/z'] > min_mz][col_ref_df[col_ref_df['row m/z'] > min_mz]['row m/z'] < max_mz]

In [ ]:
# export a dataframe to csv
df_transformed.to_csv('bucket_table-transformed.csv')